<a href="https://colab.research.google.com/github/ivynasantino/mineracao-de-dados/blob/master/06-modelo_vetorial/reports/modelo_vetorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Modelo Vetorial

Nessa atividade a ideia é implementar várias instanciações do modelo vetorial, a partir dos dados coletados do site El país.

In [102]:
# @title Imports
import pandas as pd
import math

import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
# @title Dataframe: El país
elpais = pd.read_csv("https://raw.githubusercontent.com/ivynasantino/mineracao-de-dados/master/02-processamento_de_texto/data/results.csv")

### Questões que serão abordadas:

**1.** Reconstruir o índice considerando o conjunto de dados que indicamos. Esses são os dados coletados por Bernardi e os estaremos usando para facilitar a correção da atividade. Se você já estiver usando esses dados não precisa reconstruir o índice. 

**2.** Refinar o índice invertido de forma a também incluir o IDF (inverse document frequency) de cada termo do dicionário.

**3.** Implementar as seguintes versões do modelo vetorial: 

  **a)** Representação binária
  
  **b)** TF (lembre-se que esse modelo já está implementado)
  
  **c)** TF-IDF
  
  **d)** BM25* (não usaremos Okapi já que os documentos não tem grande variação de tamanho)
  
**4.** Execute os algoritmos separadamente em 3 consultas de sua escolha e retorne os top-5 documentos mais similares à cada consulta.

**5.** Compare os resultados encontrados e responda.

  **a)** Quais modelos você acha que trouxe os melhores resultados? Por que? Inspecione os documentos retornados para melhor embasar sua resposta.

  **b)** Calcule e reporte o overlap par-a-par entre os resultados de cada modelo ([usando o índice de Jaccard](https://en.wikipedia.org/wiki/Jaccard_index))

A princípio iremos criar algumas funções auxiliares para facilitar o trabalho posteriomente.

In [0]:
def num_of_docs():
  return len(elpais['text'])

In [0]:
def top5(r):
  return sorted(r, key = lambda x: x[1], reverse = True)[:5]

### Redefinindo índice invertido

Nesse notebook, a ideia da construção do índice invertido, além dos parâmetros de chave e frequência dos documentos, iremos construir o idf.

Para calcular o novo parâmetro, seguimos o seguinte cálculo:

> **IDF = log(M+1) / k**

- M: total de documentos na coleção
- k: frequência do documento


---

Dessa maneira, fazemos esse procedimento logo abaixo:

In [0]:
toker = RegexpTokenizer(r'\w+')
stopwords = stopwords.words("portuguese") 

def buildIndex():
  index = {}
  n = 0
  for doc in elpais.text:
    n += 1
    tokens = [token for token in toker.tokenize(doc.lower()) 
              if token not in stopwords 
              and len(token) > 3
              and not bool(re.search(r'\d', token))]
 
    for t in tokens:
      freq = tokens.count(t)
      if t not in index:
        index[t] = {}
      if n not in index[t]:
        index[t][n] = freq
  return index

In [0]:
index = buildIndex()

In [0]:
for post in index:
  k = len(index[post])
  idf = math.log(num_of_docs() + 1) / k
  index[post]['idf'] = round(idf, 3)

In [0]:
vocabulary = index.keys()

Agora, nosso objetivo é implementar as versões do modelo vetorial para:

- Representação binária
- TF
- TF-IDF
- BM25

Para o modelo binário, iremos construir um vetor preenchidos com zeros e de acordo com a presença do termo é setado para um.

In [0]:
toker = RegexpTokenizer(r'\w+')
stopwords = stopwords.words("portuguese") 

def binary_rep(query):
  terms = query.split()
  tokens = [token for token in toker.tokenize(doc.lower()) 
              if token not in stopwords 
              and len(token) > 3
              and not bool(re.search(r'\d', token))]

  q = {}
  d = {}
  
  for term in terms:
    q[term], d[term] = 0,0
    if term in vocabulary:
      q[term] = 1
    if term in tokens:
      d[term] = 1
  
  m = 0
  for term in terms:
    if q[term] != 0 and d[term] != 0:
      m += q[term] * d[term]
  return m

A segunda versão, temos tf

In [0]:
toker = RegexpTokenizer(r'\w+')
stopwords = stopwords.words("portuguese") 

def tf_rep(query):
  terms = query.split()
  tokens = [token for token in toker.tokenize(doc.lower()) 
              if token not in stopwords 
              and len(token) > 3
              and not bool(re.search(r'\d', token))]
  
  q = {}
  d = {}
  
  for term in terms:
    q[term], d[term] = 0,0
    if term in vocabulary:
      q[term] = terms.count(term)
      
    if term in tokens:
      d[term] = tokens.count(term)
  
  m = 0
  for term in terms:
    if q[term] != 0 and d[term] != 0:
      m += q[term] * d[term]
      
  return m

Como terceiro ponto, temos a construção do tf idf

In [0]:
toker = RegexpTokenizer(r'\w+')
stopwords = stopwords.words("portuguese") 

def tf_idf_rep(query):
  terms = query.split()
  tokens = [token for token in toker.tokenize(doc.lower()) 
              if token not in stopwords 
              and len(token) > 3
              and not bool(re.search(r'\d', token))]
  
  q = {}
  d = {}
  
  for term in terms:
    q[term], d[term] = 0,0
    if term in vocabulary:
      q[term] = terms.count(term)
      
    if term in tokens:
      d[term] = tokens.count(term)
  
  m = 0
  for term in terms:
    idf = index[term]['idf']
    if q[term] != 0 and d[term] != 0:
      m += q[term] * d[term] * idf
      
  return round(m , 2)

Por fim, temos a versão de representação do BM25

In [0]:
toker = RegexpTokenizer(r'\w+')
stopwords = stopwords.words("portuguese") 

def bm25_rep(query, k):
  terms = query.split()
  tokens = [token for token in toker.tokenize(doc.lower()) 
              if token not in stopwords 
              and len(token) > 3
              and not bool(re.search(r'\d', token))]
  mtd = []
  for term in terms:
    if term in tokens:
      mtd.append(term)
  
  m = 0
  for mt in mtd:
    cq = terms.count(mt)
    cd = tokens.count(mt)
    m = num_of_docs()
    dw = len(index[mt].keys()) - 1
    m += cq * (((k + 1) * cd) / (cd + k)) * math.log((m + 1) / dw)
    
  return round(m, 2)

Feita as versões de modelos vetoriais, agora iremos executá-los separadamente nas consultas a seguir:

In [0]:
query = ['jair bolsonaro', 'lula livre', 'educação']

Criada a consulta, o objetivo agora é retornar o top 5 dos documentos mais similares a cada termo.

In [0]:
data = {
    'query': [],
    'binary': [],
    'tf': [],
    'tf_idf': [],
    'bm25': []
}

for q in query:
  binary, tf, tf_idf, bm25 = [], [], [], []
  n = 0
  
  for doc in elpais.text:
    binary.append((n ,binary_rep(q)))
    tf.append((n, tf_rep(q)))
    tf_idf.append((n, tf_idf_rep(q)))
    bm25.append((n, bm25_rep(q, 10)))
    n += 1

  data['query'].append(q)
  data['binary'].append(top5(binary))
  data['tf'].append(top5(tf))
  data['tf_idf'].append(top5(tf_idf))
  data['bm25'].append(top5(bm25))

Convertendo para um dataframe, facilitando assim a visualização, temos:

In [95]:
pd.options.display.max_colwidth = 100
pd.DataFrame(data)

,query,binary,tf,tf_idf,bm25
0,jair bolsonaro,"[(0, 2), (1, 2), (24, 2), (85, 2), (125, 2)]","[(150, 52), (206, 48), (165, 39), (18, 26), (215, 13)]","[(206, 6.42), (150, 5.38), (165, 3.55), (18, 2.11), (215, 1.25)]","[(150, 260.9), (165, 260.4), (206, 260.27), (18, 259.46), (41, 256.9)]"
1,lula livre,"[(25, 2), (171, 2), (237, 2), (4, 1), (14, 1)]","[(14, 9), (210, 3), (215, 3), (233, 3), (25, 2)]","[(14, 3.82), (215, 1.27), (233, 1.27), (167, 0.85), (203, 0.85)]","[(14, 264.82), (215, 256.71), (233, 256.71), (210, 254.75), (167, 254.57)]"
2,educação,"[(2, 1), (17, 1), (31, 1), (36, 1), (86, 1)]","[(220, 22), (221, 11), (129, 7), (238, 6), (36, 5)]","[(220, 4.18), (221, 2.09), (129, 1.33), (238, 1.14), (36, 0.95)]","[(220, 265.56), (221, 261.61), (129, 258.92), (238, 258.03), (36, 257.03)]"


A partir da tabela acima, podemos intuitivamente concluir qual das versões de modelo vetorial tiveram melhores resultados. E como resposta, o modelo BM25 consegue equilibrar entre penalização e recompensa dos documentos e, no "olhômetro" houve uma pequena variação de score, dessa forma, podemos dizer que, empiricamente, que possa ser um bom canditado a melhor modelo, já que existem métricas para esse tipo de medição, no entanto não estão sendo utilizadas nesse notebbok.